# Scraping

In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
from urllib.parse import urlsplit
import requests
import pandas as pd
import os
import time
import shutil
from IPython.display import Image

In [2]:
# Import Splinter and set the chromedriver path
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

# NASA Mars News

In [3]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
news_title = soup.find('div', attrs={'class':'content_title'})
news_paragraph = soup.find('div', attrs={'class':'article_teaser_body'})
print(f"Title: {news_title}")
print(f"Paragraph: {news_paragraph}")

Title: <div class="content_title">Mars Now</div>
Paragraph: <div class="article_teaser_body">Vast areas of the Martian night sky pulse in ultraviolet light, according to images from NASA’s MAVEN spacecraft. The results are being used to illuminate complex circulation patterns in the Martian atmosphere.</div>


# JPL Mars Space Images - Featured Image

In [5]:
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=featured#submit"
browser.visit(url_image)

In [6]:
url_html = browser.html
soup = BeautifulSoup(url_html, 'html.parser')
fancybox = soup.find('a', class_ = 'button fancybox')
img_link = fancybox['data-link']

In [7]:
browser.visit(f'https://www.jpl.nasa.gov{img_link}')
image_url = browser.html
soup_ = BeautifulSoup(image_url, 'html.parser')

In [9]:
full_image_url = soup_.find('img', class_ = "main_image")['src']
full_image_url = (f'https://www.jpl.nasa.gov{full_image_url}')
full_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17767_hires.jpg'

# Mars Weather

In [10]:
url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)
html = browser.html

In [11]:
tweets = soup.find_all(attrs={"data-testid": "tweet"})
spans = soup.find_all('span', class_='css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0')
mars_weather = (tweets, spans)
print(f"Title: {tweets}")
print(f"Para: {spans}")

Title: []
Para: []


# Mars Facts

In [12]:
mars_facts_url = "https://space-facts.com/mars/"
table = pd.read_html(mars_facts_url)
table[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [13]:
df_mars_facts = table[0]
df_mars_facts.columns = ["Facts", "Values"]
df_mars_facts.set_index(["Facts"])

,Values
Facts,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [14]:
mars_html_table = df_mars_facts.to_html()
mars_html_table = mars_html_table.replace("\n", "")
mars_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Facts</th>      <th>Values</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</

# Mars Hemispheres

In [15]:
hemisphere_image_urls = []

In [16]:
url_hemisphere = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hemisphere)
hemisphere_base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url_hemisphere))
print(hemisphere_base_url)

https://astrogeology.usgs.gov/


Cerberus-Hemisphere

In [17]:
url_cerberus = "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"
browser.visit(url_cerberus)
response_cerberus = browser.html
soup4 = BeautifulSoup(response_cerberus, 'html.parser')
cerberus_img = soup4.find_all('div', class_="wide-image-wrapper")
for img in cerberus_img:
        pic_cerberus = img.find('li')
        cerberus_full_img = pic_cerberus.find('a')['href']
cerberus_title = soup4.find('h2', class_='title').get_text()
cerberus_hem = {"Title": cerberus_title, "url": cerberus_full_img}
hemisphere_image_urls.append(cerberus_hem)

Schiaparelli-Hemisphere

In [18]:
url_schiaparelli = "https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced"
browser.visit(url_cerberus)
response_schiaparelli = browser.html
soup5 = BeautifulSoup(response_schiaparelli, 'html.parser')
schiaparelli_img = soup5.find_all('div', class_="wide-image-wrapper")
for img in schiaparelli_img:
        pic_schiaparelli = img.find('li')
        schiaparelli_full_img = pic_schiaparelli.find('a')['href']
shiaparelli_title = soup5.find('h2', class_='title').get_text()
shiaparelli_hem = {"Title": shiaparelli_title, "url": schiaparelli_full_img}
hemisphere_image_urls.append(shiaparelli_hem)

Syrtis Major Hemisphere

In [19]:
url_syrtis = ('https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced')
browser.visit(url_syrtis)
response_syrtis = browser.html
soup6 = BeautifulSoup(response_syrtis, 'html.parser')
syrtris_img = soup6.find_all('div', class_="wide-image-wrapper")
for img in syrtris_img:
        pic_syrtris = img.find('li')
        syrtris_full_img = pic_syrtris.find('a')['href']
syrtris_title = soup6.find('h2', class_='title').get_text()
syrtris_hem = {"Title": syrtris_title, "url": syrtris_full_img}
hemisphere_image_urls.append(syrtris_hem)

Valles Marineris Hemisphere

In [20]:
url_valles = ('https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced')
browser.visit(url_valles)
response_valles = browser.html
soup7 = BeautifulSoup(response_valles, 'html.parser')
valles_img = soup7.find_all('div', class_="wide-image-wrapper")
for img in valles_img:
        pic_valles = img.find('li')
        valles_full_img = pic_valles.find('a')['href']
valles_title = soup7.find('h2', class_='title').get_text()
valles_hem = {"Title": valles_title, "url": valles_full_img}
hemisphere_image_urls.append(valles_hem)

In [21]:
hemisphere_image_urls

[{'Title': 'Cerberus Hemisphere Enhanced',
  'url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'Title': 'Cerberus Hemisphere Enhanced',
  'url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'Title': 'Syrtis Major Hemisphere Enhanced',
  'url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'Title': 'Valles Marineris Hemisphere Enhanced',
  'url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [22]:
browser.quit()